In [ ]:
import os
import zipfile

local_zip='/content/dataset.zip'
zip_ref=zipfile.ZipFile(local_zip,'r')
zip_ref.extractall('/content/')
zip_ref.close()

In [ ]:
import tensorflow as tf
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.preprocessing.image import ImageDataGenerator


In [ ]:
train_data_gen=ImageDataGenerator(rescale=1/255,
                                  rotation_range=30,
                                  height_shift_range=0.2,
                                  width_shift_range=0.2,
                                  shear_range=0.6,
                                  zoom_range=0.2,
                                  horizontal_flip=True,
                                  fill_mode='nearest')
test_data_gen=ImageDataGenerator(rescale=1/255)

In [ ]:
import pandas as pd
train_df=pd.read_csv('/content/dataset/train.csv')
test_df=pd.read_csv('/content/dataset/test.csv')


In [ ]:
test_df.head()

,Image
0,508.jpg
1,246.jpg
2,473.jpg
3,485.jpg
4,128.jpg


In [ ]:
train_generator=train_data_gen.flow_from_dataframe(dataframe=train_df,directory='/content/dataset/train',
                                                   x_col='Image',
                                                   y_col='target',
                                                   target_size=(224,224),
                                                   class_mode='categorical',
                                                   batch_size=32,shuffle=True)

test_generator=test_data_gen.flow_from_dataframe(dataframe=test_df,directory='/content/dataset/test',
                                                 x_col='Image',
                                                 y_col=None,
                                                   target_size=(224,224),
                                                   class_mode=None,
                                                   batch_size=32,shuffle=False)

Found 364 validated image filenames belonging to 8 classes.
Found 156 validated image filenames.


In [ ]:
import tensorflow_datasets as tfds

In [ ]:
import tensorflow_hub as hub

URL = "https://tfhub.dev/google/tf2-preview/mobilenet_v2/feature_vector/4" #https://tfhub.dev/google/tf2-preview/inception_v3/feature_vector/4
feature_extractor = hub.KerasLayer(URL, input_shape=(224, 224 ,3))
feature_extractor.trainable = False



In [ ]:
model=tf.keras.Sequential([
                           feature_extractor,
                           tf.keras.layers.Dense(8,activation='softmax')
])
model.summary()

Model: "sequential_10"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
keras_layer (KerasLayer)     (None, 1280)              2257984   
_________________________________________________________________
dense_15 (Dense)             (None, 8)                 10248     
Total params: 2,268,232
Trainable params: 10,248
Non-trainable params: 2,257,984
_________________________________________________________________


In [ ]:
model.compile(optimizer=tf.optimizers.Adam(),loss='categorical_crossentropy',metrics=['accuracy'])

In [ ]:
history = model.fit(train_generator , epochs=100 ,steps_per_epoch=train_generator.n//train_generator.batch_size)

Epoch 1/100
11/11 [==============================] - 4s 404ms/step - loss: 2.4555 - accuracy: 0.1145
Epoch 2/100
11/11 [==============================] - 4s 404ms/step - loss: 2.3071 - accuracy: 0.1295
Epoch 3/100
11/11 [==============================] - 5s 411ms/step - loss: 2.1549 - accuracy: 0.1717
Epoch 4/100
11/11 [==============================] - 4s 406ms/step - loss: 2.1173 - accuracy: 0.1867
Epoch 5/100
11/11 [==============================] - 4s 394ms/step - loss: 2.0971 - accuracy: 0.2078
Epoch 6/100
11/11 [==============================] - 4s 405ms/step - loss: 2.0150 - accuracy: 0.2289
Epoch 7/100
11/11 [==============================] - 4s 398ms/step - loss: 1.9767 - accuracy: 0.2440
Epoch 8/100
11/11 [==============================] - 5s 442ms/step - loss: 1.9047 - accuracy: 0.3102
Epoch 9/100
11/11 [==============================] - 4s 394ms/step - loss: 1.8459 - accuracy: 0.3343
Epoch 10/100
11/11 [==============================] - 4s 401ms/step - loss: 1.8635 - accura

In [ ]:
print(train_generator.class_indices.items())

dict_items([('bharatanatyam', 0), ('kathak', 1), ('kathakali', 2), ('kuchipudi', 3), ('manipuri', 4), ('mohiniyattam', 5), ('odissi', 6), ('sattriya', 7)])


In [ ]:
test_df['target']=[list(train_generator.class_indices.items())[i][0] for i in model.predict_classes(test_generator).tolist()]

In [ ]:
test_df.head()

,Image,target
0,508.jpg,kuchipudi
1,246.jpg,odissi
2,473.jpg,bharatanatyam
3,485.jpg,kuchipudi
4,128.jpg,bharatanatyam


In [ ]:
test_df.to_csv('/content/submission.csv',index=False)